In [ ]:
import xml.etree.ElementTree as ET
import cv2
from PIL import Image
from os import listdir as ld
from os.path import join as pj
import numpy as np
from scipy import ndimage
import h5py
from scipy import sparse
from tqdm import tqdm

from IO.loader import parse_annotations, file_id
from evaluation.classification.statistics import compute_anno_stats, compute_average_size, compute_size_correction
from evaluation.classification.visualize import plot_size_of_anno, plot_size_by_class_of_anno
from utils.crop import *
from utils.annotate import *
from IO.build_ds import build_classification_ds

import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline

### Load data

In [ ]:
root = "/home/tanida/workspace/Insect_Phenology_Detector/data"
anno_folders = ["annotations_20200806"]
annos = []
for anno_folder in anno_folders:
    annos_name = ld(pj(root, anno_folder))
    annos.extend([pj(root, anno_folder, x) for x in annos_name])
imgs  = ld(pj(root, "refined_images"))
imgs  = [pj(root, "refined_images", x) for x in imgs if x != ".ipynb_checkpoints"]

In [ ]:
images = {file_id(im):np.array(Image.open(im)) for im in imgs}
annotations = {idx: list(filter(lambda x:idx in x, annos)) for idx in images}
annotations = {k:v for  k,v in annotations.items() if len(v)>0}

anno = {}
for k,v in annotations.items():
    anno[k]=[]
    for x in filter(lambda x:x.endswith(".xml"), v):
        anno[k].extend(parse_annotations(x))

### Make datasets

In [ ]:
X,Y = build_classification_ds(anno, images, crop=crop_adjusted_std)

### Visualize

In [ ]:
lbl_to_name = {
    0: 'Coleoptera', 
    1: 'Diptera', 
    2: 'Ephemeridae', 
    3: 'Ephemeroptera', 
    4: 'Hemiptera', 
    5: 'Hymenoptera', 
    6: 'Lepidoptera', 
    7: 'Megaloptera', 
    8: 'Plecoptera', 
    9: 'Trichoptera', 
    10: 'Unknown', 
    11: ']', 
    12: 'unknown'
}

In [ ]:
idxs, counts = np.unique(Y, return_counts=True)
idxs = np.asarray([lbl_to_name[idx] for idx in idxs])
for idx, count in zip(idxs, counts):
    print("{}: {}".format(idx, count))

In [ ]:
idx = 0

In [ ]:
img = X[idx]
label = lbl_to_name[Y[idx]]
plt.imshow(img)
print(label)
idx += 1